# Interfacing Julia with other languages

Several examples are taken from the "_Julia is fast_" (available [here](https://github.com/JuliaComputing/JuliaBoxTutorials/blob/master/introductory-tutorials/intro-to-julia/09.%20Julia%20is%20fast.ipynb))

## Calling C and Fortran code

https://docs.julialang.org/en/v1/manual/calling-c-and-fortran-code/

C functions can be called directly from Julia by using the `ccall` syntax, with little to no overhead.
The code must be available as a shared library.

Shared libraries and functions are referenced by a tuple of the form `(:function, "library")` or `("function", "library")`:
* `function` is the C-exported function name.
* `library` refers to the shared library name: shared libraries available in the (platform-specific) load path will be resolved by name, and if necessary a direct path may be specified.

First, we write a sample C code that computes sum of an double-valued array.

In [1]:
using Libdl
const Clib = tempname()   # make a temporary file

"/tmp/juliathZd30"

In [2]:
# Sample C code
# This function computes the sum of an array of double
C_code = """
#include <stddef.h>
double c_sum(size_t n, double *X) {
    double s = 0.0;
    for (size_t i = 0; i < n; ++i) {
        s += X[i];
    }
    return s;
}
"""
;

In [3]:
# compile to a shared library by piping C_code to gcc
# (works only if you have gcc installed):
open(`gcc -fPIC -O3 -msse3 -xc -shared -o $(Clib * "." * Libdl.dlext) -`, "w") do f
    print(f, C_code) 
end

You can use `ccall` to actually generate a call to the library function. Arguments to `ccall` are as follows:

1. A `(:function, "library")` pair, which must be written as a literal constant
2. Return type
3. A tuple of input types. The input types must be written as a literal tuple, not a tuple-valued variable or expression.
4. The following arguments, if any, are the actual argument values passed to the function.

This syntax can be heavy to read, to it is common to wrap the C function into a Julia function

In [4]:
# Wrap the C function in a Julia function
c_sum(X::Array{Float64}) = ccall(
    ("c_sum", Clib),          # The C-function's name and the library it comes from
    Float64,                  # Return type
    (Csize_t, Ptr{Float64}),  # Tuple of input types
    length(X), X              # actual arguments
)

c_sum (generic function with 1 method)

In [5]:
u = ones(10)
c_sum(u)

10.0

## Calling Python code from Julia

`PyCall` is the default package for calling Python from Julia: https://github.com/JuliaPy/PyCall.jl

>  PyCall provides the ability to directly call and fully interoperate with Python from the Julia language.

> You can import arbitrary Python modules from Julia, call Python functions (with automatic conversion of types between Julia and Python), define Python classes from Julia methods, and share large data structures between Julia and Python without copying them.

In [6]:
# using Pkg; Pkg.add("PyCall")
using PyCall

In [7]:
py_sum = pybuiltin("sum")  # Python's built-in `sum` function
py_sum(u)

10.0

In [8]:
# using Pkg; Pkg.add("Conda")  # un-comment this line to install Conda
using Conda

In [9]:
# Conda.add("numpy")  # un-comment this line to install numpy

In [10]:
np_sum = pyimport("numpy")["sum"]  # Numpy's `sum` function, i.e., `numpy.sum`
np_sum(u)

10.0

## Calling other languages (C++, R, Java, etc)

* Calling C++ from Julia
    * `Cxx` : https://github.com/Keno/Cxx.jl
    * `CxxWrap` : https://github.com/JuliaInterop/CxxWrap.jl


* Calling Java from Julia: `JavaCall` https://github.com/JuliaInterop/JavaCall.jl


* Calling R from Julia: `RCall` https://github.com/JuliaInterop/RCall.jl


* Calling other languages from Julia: https://github.com/JuliaInterop

# Embedding Julia

## The C API

## Calling Julia from Python